# Automated Response
- **Objective:** To generate alerts when certain fault conditions occur

In [12]:
# Import libraries
import pandas as pd  # Data tables
import seaborn as sns  # Create charts
import matplotlib.pyplot as plt  # Display charts
from sklearn.model_selection import train_test_split  # Split data for training and testing
from sklearn.preprocessing import LabelEncoder  # Convert text to numbers
from sklearn.tree import DecisionTreeClassifier  # Machine learning model
from sklearn.metrics import classification_report, confusion_matrix  # Check how well the model works

In [13]:
# Read data file
df = pd.read_csv('lighting_data_cleaned.csv')

# Display data
df

,light_id,location_name,fault_type,timestamp,severity_level,fault_status,maintenance_cost,year,month,day,day_of_week,hour
0,L0226,Pasir Ris,Control System,2020-01-01 01:22:25,High,Acknowledged,343.60,2020,1,1,Wednesday,1
1,L0061,City Hall,Control System,2020-01-01 23:45:18,Low,In Progress,271.80,2020,1,1,Wednesday,23
2,L0455,Pasir Ris,Power-Related,2020-01-02 01:33:02,Informational,Resolved,169.20,2020,1,2,Thursday,1
3,L0924,City Hall,Cybersecurity,2020-01-02 03:34:48,Informational,Resolved,372.45,2020,1,2,Thursday,3
4,L0134,Canberra,Communication,2020-01-02 08:09:16,Low,In Progress,335.57,2020,1,2,Thursday,8
...,...,...,...,...,...,...,...,...,...,...,...,...
461,L0835,Orchard Road,Sensor-Related,2020-12-24 23:46:51,Medium,Acknowledged,482.02,2020,12,24,Thursday,23
462,L0452,Jurong West,Environmental,2020-12-25 02:25:42,Informational,Detected,277.74,2020,12,25,Friday,2
463,L0483,Bishan,Sensor-Related,2020-12-25 02:34:36,Critical,In Progress,198.08,2020,12,25,Friday,2
464,L0683,Canberra,Control System,2020-12-29 20:00:19,Critical,In Progress,236.10,2020,12,29,Tuesday,20


In [14]:
# Filter faults with severity High or Critical
alert_faults = df[df['severity_level'].isin(['High', 'Critical'])]

# Filter the DataFrame 'df' to include only rows where:
# 1) The 'fault_type' column is exactly 'Power-related'
# AND
# 2) The 'severity_level' column value is either 'Medium', 'High', or 'Critical'
power_alerts = df[
    (df['fault_type'] == 'Power-related') &  # Condition 1: fault_type must be 'Power-related'
    (df['severity_level'].isin(['Medium', 'High', 'Critical']))  # Condition 2: severity_level must be one of these values
]

# Combine the two filtered DataFrames 'alert_faults' and 'power_alerts' into one DataFrame,
# then remove any duplicate rows that might appear in both,
# and finally reset the index of the resulting DataFrame for clean numbering.
all_alerts = (
    pd.concat([alert_faults, power_alerts])  # Concatenate the two DataFrames vertically (stack rows)
    .drop_duplicates()                      # Remove any duplicate rows that exist in both DataFrames
    .reset_index(drop=True)                 # Reset the index to have a clean sequential index starting at 0
)

# Print the total number of alert records in the 'all_alerts' DataFrame
print(f"Total alerts generated: {len(all_alerts)}")

# Example alert display
for idx, row in all_alerts.iterrows():
    print(f"ALERT: Fault '{row['fault_type']}' at '{row['location_name']}' with severity '{row['severity_level']}' on {row['timestamp']}")

Total alerts generated: 199
ALERT: Fault 'Control System' at 'Pasir Ris' with severity 'High' on 2020-01-01 01:22:25
ALERT: Fault 'Cybersecurity' at 'Bugis Junction' with severity 'High' on 2020-01-03 19:12:37
ALERT: Fault 'Control System' at 'Pasir Ris' with severity 'Critical' on 2020-01-04 12:10:28
ALERT: Fault 'Environmental' at 'Bishan' with severity 'Critical' on 2020-01-05 21:22:55
ALERT: Fault 'Environmental' at 'Bishan' with severity 'Critical' on 2020-01-08 10:07:10
ALERT: Fault 'Cybersecurity' at 'Bugis Junction' with severity 'Critical' on 2020-01-11 05:37:07
ALERT: Fault 'Control System' at 'Bugis Junction' with severity 'Critical' on 2020-01-11 07:03:02
ALERT: Fault 'Power-Related' at 'Chinatown' with severity 'Critical' on 2020-01-14 01:35:02
ALERT: Fault 'Environmental' at 'Bishan' with severity 'High' on 2020-01-17 23:13:13
ALERT: Fault 'Control System' at 'Canberra' with severity 'High' on 2020-01-18 18:28:01
ALERT: Fault 'Environmental' at 'Toa Payoh' with severity '

In [15]:
# Create a new DataFrame with only selected columns from all_alerts
styled_alerts = all_alerts[['fault_type', 'location_name', 'severity_level', 'timestamp']].copy()

# Define severity levels colours
severity_colors = {
    "Critical": "background-color: #ff4d4d; color: white; font-weight: bold;", # Red
    "High": "background-color: #ffa500; color: white; font-weight: bold;", # Orange
    "Medium": "background-color: #ffeb3b; color: black;" # Yellow
}

# Return colour for the given severity level
def highlight_severity(val):
    return severity_colors.get(val, "")

# Apply color styling to the severity_level column and set table styles
styled_alerts.style.applymap(highlight_severity, subset=['severity_level']) \
    .set_table_styles([
        {'selector': 'thead th', 'props': [('background-color', '#333'), ('color', 'white'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('border', '1px solid #ccc'), ('padding', '6px')]},
    ]) \
    .set_properties(**{'text-align': 'center'})

/tmp/ipython-input-1786090863.py:16: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_alerts.style.applymap(highlight_severity, subset=['severity_level']) \


,fault_type,location_name,severity_level,timestamp
0,Control System,Pasir Ris,High,2020-01-01 01:22:25
1,Cybersecurity,Bugis Junction,High,2020-01-03 19:12:37
2,Control System,Pasir Ris,Critical,2020-01-04 12:10:28
3,Environmental,Bishan,Critical,2020-01-05 21:22:55
4,Environmental,Bishan,Critical,2020-01-08 10:07:10
5,Cybersecurity,Bugis Junction,Critical,2020-01-11 05:37:07
6,Control System,Bugis Junction,Critical,2020-01-11 07:03:02
7,Power-Related,Chinatown,Critical,2020-01-14 01:35:02
8,Environmental,Bishan,High,2020-01-17 23:13:13
9,Control System,Canberra,High,2020-01-18 18:28:01
